<a href="https://colab.research.google.com/github/neemiasbsilva/MLLMs-Teoria-e-Pratica/blob/main/use-cases/FineTuning_ModernBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning do ModernBERT para Classificação de Sentimento

Este notebook guia você pelo processo de fine-tuning de um modelo textual (`answerdotai/ModernBERT-large`) para uma tarefa de classificação de sentimento. Usaremos os scripts fornecidos, adaptados para o ambiente Colab, e adicionaremos o monitoramento de métricas com o TensorBoard.

Para mais informações pode consultar o pipeline original no seguinte repositório: [MLLMsent-framework](https://github.com/neemiasbsilva/MLLMsent-framework/tree/main).

## Configuração do Ambiente

Primeiro, vamos instalar as bibliotecas necessárias.

- `transformers`, `datasets`, `peft`, `trl`: Essenciais para lidar com os modelos.
- `scikit-learn`: Para métricas.
- `pyaml`: Para carregar o arquivo de configuração.
- `gdown`: Para baixar o dataset do Google Drive.
- `tensorboard`: Para o monitoramente.

In [ ]:
!pip install transformers[torch] datasets scikit-learn pyyaml gdown tensorboard -q

## Importações dos Pacotes

Agora, vamos importar todos os módulos que usaremos no projeto.

In [ ]:
import torch
import os
import sys
import time
import random
import pandas as pd
import numpy as np
import warnings
import yaml
import gdown
from tqdm.notebook import tqdm
from scipy import stats
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter  # Importa o TensorBoard
from torch.optim import AdamW

# Imports da Hugging Face
from transformers import (
    AutoModel,
    AutoTokenizer,
    TrainingArguments
)

# Imports do Scikit-learn
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Criação do Arquivo de Configuração

Vamos criar o arquivo `config.yaml`e os diretórios necessários para os logs e checkpoints, conforme especificado.


In [ ]:
# Criar os diretórios
!mkdir -p experiments-finetuning/openai-modernbert-experiment-p3-alpha3/logs
!mkdir -p checkpoints

# Definir o conteúdo do YAML
config_content = """
experiment_name: "Experiment Using Moder BERT"
learning_rate: 1e-5
batch_size: 4
epochs: 5
model_path: "answerdotai/ModernBERT-large"
max_len: 512
model_name: "modern-bert"
log_dir: "experiments-finetuning/openai-modernbert-experiment-p3-alpha3/logs"
checkpoint_dir: "checkpoints"
"""

# Escrever o arquivo config.yaml
config_path = "experiments-finetuning/openai-modernbert-experiment-p3-alpha3/config.yaml"
with open(config_path, "w") as f:
    f.write(config_content)

# Função para carregar o config (de utils.other_utils)
def load_config(config_path):
    """Carrega o arquivo de configuração YAML."""
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
    print(f"Configuração carregada de: {config_path}")
    return config

# Carregar a configuração para verificar
config = load_config(config_path)
print(config)

## Carregamento e Preparação dos Dados

Vamos baixar o dataset do Google Drive e prepará-lo. O Script `train` espera uma estrutura de pastas específicas (`data/{dataset_type}/...`). Vamos criar essa estrutura e baixar o arquivo para o local correto.



In [ ]:
# Criar a estrutura de diretório que o script espera
# Com base no config_path, o script vai gerar:
# dataset_type = "gpt4-openai-classify"
# alpha_version = 3
# experiment_group = "p3"
# Path final: "data/gpt4-openai-classify/percept_dataset_alpha3_p3.csv"

data_dir = "data/gpt4-openai-classify"
data_file_path = os.path.join(data_dir, "percept_dataset_alpha3_p3.csv")
!mkdir -p {data_dir}

# Baixar o arquivo do Google Drive
gdrive_url = "https://drive.google.com/file/d/1a2XrWeXHTjLR_5zWoiK4tsIY-p23iqxO/view?usp=share_link"
gdrive_id = "1a2XrWeXHTjLR_5zWoiK4tsIY-p23iqxO"
print(f"Baixando dataset para: {data_file_path}")
gdown.download(id=gdrive_id, output=data_file_path, quiet=False)

# Funções de carregamento de dados (do script)
def load_dataframe(file_path):
    """Carrega um dataset de um arquivo CSV."""
    try:
        df = pd.read_csv(file_path)
        print(f"Dataset carregado com sucesso de: {file_path}")
        return df
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado em {file_path}")
        return None

def load_experiment_data(alpha_version, dataset_type, experiment_group):
    """Carrega o dataset dinamicamente."""
    file_path = f"data/{dataset_type}/percept_dataset_alpha{alpha_version}_{experiment_group}.csv"
    df = load_dataframe(file_path)
    return df

## Definição do DataLoader

Vamos criar uma implementação padrão do PyTorch (Dataset + DataLoader) que se encaixa no loop de treino.

In [ ]:
class CustomSentimentDataset(Dataset):
    """Dataset customizado para classificação de sentimento."""
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text.values
        self.targets = dataframe.sentiment.values
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )

        return {
            'ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'token_type_id': torch.tensor(inputs['token_type_ids'], dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)
        }

def data_loader(df, tokenizer, max_len, params):
    """
    Cria uma instância do DataLoader.
    """
    dataset = CustomSentimentDataset(df, tokenizer, max_len)
    return DataLoader(dataset, **params)

## Definição do Modelo

In [ ]:
class ModernBERTModel(torch.nn.Module):
    """Classe do modelo ModernBERT com uma cabeça de classificação customizada."""
    def __init__(self, model_id, class_size):
        super().__init__()
        self.model_id = model_id
        self.model = AutoModel.from_pretrained(model_id)

        # Adiciona um classificador customizado
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(self.model.config.hidden_size, 1024),
            torch.nn.ReLU(),
            torch.nn.Linear(1024, class_size),
        )

    def forward(self, ids, mask, token_type_id=None):
        # Passa a entrada pelo modelo pré-treinado
        # O ModernBERT (baseado em BERT) aceita token_type_ids
        output = self.model(ids, attention_mask=mask)
        last_hidden_state = output.last_hidden_state

        # Usa o token [CLS] (posição 0) para classificação
        CLS_token_state = last_hidden_state[:, 0, :]
        out = self.classifier(CLS_token_state)

        return out

## Funções Uteis de Treino e Avaliação

In [ ]:
# Funções de cálculo de métricas e perdas
def compute_loss(outputs, targets, loss_fn, model_name):
    return loss_fn(outputs, targets)

def compute_metrics(preds, targets):
    accuracy = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average="weighted")
    return accuracy, f1

# Funções de loop de época (train/val)

def train_one_epoch(model, train_dl, optimizer, loss_fn, device, model_name):
    model.train()
    total_loss = 0.0
    preds, targets = [], []

    for _, data in enumerate(tqdm(train_dl, desc="Treinando Época", leave=False)):
        ids = data["ids"].to(device)
        mask = data["mask"].to(device)
        targets_batch = data["targets"].to(device)
        # token_type_id = data["token_type_id"].to(device) # Removido

        optimizer.zero_grad()
        # outputs = model(ids, mask, token_type_id) # Original
        outputs = model(ids, mask) # Modificado

        loss = compute_loss(outputs, targets_batch, loss_fn, model_name)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds.extend(torch.argmax(outputs, axis=1).tolist())
        targets.extend(targets_batch.tolist())

    return total_loss, preds, targets

def validate_one_epoch(model, val_dl, loss_fn, device, model_name):
    model.eval()
    total_loss = 0.0
    preds, targets = [], []

    with torch.no_grad():
        for _, data in enumerate(tqdm(val_dl, desc="Validando", leave=False)):
            ids = data["ids"].to(device)
            mask = data["mask"].to(device)
            targets_batch = data["targets"].to(device)
            # token_type_id = data["token_type_id"].to(device) # Removido

            # outputs = model(ids, mask, token_type_id) # Original
            outputs = model(ids, mask) # Modificado

            loss = compute_loss(outputs, targets_batch, loss_fn, model_name)

            total_loss += loss.item()
            preds.extend(torch.argmax(outputs, axis=1).tolist())
            targets.extend(targets_batch.tolist())

    return total_loss, preds, targets

# Funções de logging e checkpoint
def save_checkpoint(model, checkpoint_dir, name_arch, experiment_group, alpha_version, fine_tuning, f1_val, best_f1score):
    if f1_val > best_f1score:
        best_f1score = f1_val
        checkpoint_path = os.path.join(
            checkpoint_dir, f"best_checkpoint_{name_arch.split('/')[0]}_{experiment_group}_sigma{alpha_version}_{fine_tuning}.pt"
        )
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Checkpoint salvo em: {checkpoint_path} com F1-score: {f1_val:.4f}")
    return best_f1score

def log_metrics(epoch, epochs, train_loss, train_accuracy, train_f1, val_loss, val_accuracy, val_f1, log_file):
    log_entry = (
        f"Epoch {epoch+1}/{epochs}: \n"
        f"Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, "
        f"F1-score: {train_f1:.4f}\n"
        f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, "
        f"F1-score: {val_f1:.4f}\n"
    )
    print(log_entry) # Imprime no console do Colab
    with open(log_file, "a") as f:
        f.write(log_entry)

# Funções de validação final (pós-treino)
def compute_val_loss_and_preds(model, dataloader, loss_fn, device, model_name):
    total_loss = 0.0
    preds, targets = [], []
    model.eval() # Garante que o modelo está em modo de avaliação
    with torch.no_grad():
        for _, data in enumerate(dataloader):
            ids = data["ids"].to(device)
            mask = data["mask"].to(device)
            targets_batch = data["targets"].to(device)
            # token_type_id = data["token_type_id"].to(device) # Removido

            # outputs = model(ids, mask, token_type_id) # Original
            outputs = model(ids, mask) # Modificado

            loss = loss_fn(outputs, targets_batch)
            total_loss += loss.item()
            preds.extend(torch.argmax(outputs, axis=1).tolist())
            targets.extend(targets_batch.tolist())
    return total_loss, preds, targets

def compute_val_metrics(preds, targets, total_loss, dataloader):
    accuracy = accuracy_score(targets, preds)
    f1 = f1_score(targets, preds, average="weighted")
    loss = total_loss / len(dataloader)
    return accuracy, f1, loss

def update_metrics_df(df_metrics, kfold, accuracy, f1, start_time):
    new_metrics = pd.DataFrame({
        "kfold": [kfold + 1],
        "accuracy": [accuracy],
        "f1_score": [f1],
        "time": [int(time.time()-start_time)]
    })
    df_metrics = pd.concat([df_metrics, new_metrics], axis=0)
    return df_metrics

def save_metrics_to_csv(df_metrics, log_dir):
    df_metrics.to_csv(os.path.join(log_dir, f"test_logs.csv"), index=False)

## Função `fit` (Treino) com TensorBoard

Esta é a função `fit`principal, modificada para inicializar e usar o `SummaryWriter` do TensorBoard para logar as métricas a cada época.

In [ ]:
def freeze_backbone(model, head_names=("classifier", "score", "lm_head", "classification_head")):
    """Congela todos os parâmetros, exceto os da camada 'head'."""
    for child_name, child_module in model.named_children():
        if child_name in head_names:
            print(f"Camada '{child_name}' NÃO foi congelada.")
            continue
        # Congela este submódulo
        for p in child_module.parameters():
            p.requires_grad = False

def fit(
    model, class_weights, epochs, optimizer,
    train_dl, val_dl,
    log_dir, checkpoint_dir,
    name_arch, fold, model_name,
    alpha_version, experiment_group,
    device
):
    # Inicializa o TensorBoard Writer para este fold
    tb_log_dir = os.path.join(log_dir, f"fold_{fold+1}")
    writer = SummaryWriter(log_dir=tb_log_dir)
    print(f"Logs do TensorBoard para este fold: {tb_log_dir}")

    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights, reduction="mean")

    patience = 10
    if log_dir.split('/')[0] == "experiments-not-finetuning":
        print("Backbone freeze")
        patience = 25
        freeze_backbone(model)

    torch.manual_seed(42)
    np.random.seed(42)

    log_file = os.path.join(log_dir, f"training_logs_{fold+1:02d}.txt")
    open(log_file, 'w').close()

    df_metrics = pd.DataFrame([])
    best_f1score = 0
    patience_counter = 0

    for epoch in range(epochs):
        print(f"\n--- Época {epoch+1}/{epochs} ---")
        # Fase de Treinamento
        train_loss, preds_train, targets_train = train_one_epoch(model, train_dl, optimizer, loss_fn, device, model_name)
        accuracy_train, f1_train = compute_metrics(preds_train, targets_train)
        train_loss /= len(train_dl)

        # Fase de Validação
        val_loss, preds_val, targets_val = validate_one_epoch(model, val_dl, loss_fn, device, model_name)
        accuracy_val, f1_val = compute_metrics(preds_val, targets_val)
        val_loss /= len(val_dl)

        # Logar métricas no arquivo de texto
        log_metrics(epoch, epochs, train_loss, accuracy_train, f1_train, val_loss, accuracy_val, f1_val, log_file)

        # === ADIÇÃO DO TENSORBOARD ===
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', accuracy_train, epoch)
        writer.add_scalar('F1/train', f1_train, epoch)
        writer.add_scalar('Loss/validation', val_loss, epoch)
        writer.add_scalar('Accuracy/validation', accuracy_val, epoch)
        writer.add_scalar('F1/validation', f1_val, epoch)
        # ==============================

        # Salvar checkpoint (lógica original)
        if f1_val > best_f1score:
            best_f1score = f1_val
            # (A lógica de salvar o checkpoint foi movida para a função `train` principal)
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"Paciência: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print(f"Parada antecipada: F1-score de validação não melhorou por {patience} épocas.")
            break

        # Atualizar DataFrame de métricas
        df_metrics = pd.concat(
            [df_metrics, pd.DataFrame({
                "epoch": [epoch + 1],
                "train_accuracy": [accuracy_train],
                "train_f1_score": [f1_train],
                "val_accuracy": [accuracy_val],
                "val_f1_score": [f1_val],
            })],
            axis=0
        )
        df_metrics.to_csv(os.path.join(log_dir, f"training_logs_{fold+1:02d}.csv"), index=False)

    writer.close() # Fecha o writer do TensorBoard
    return model, loss_fn

def val(log_dir, model, dataloader, loss_fn, kfold, df_metrics, model_name, device, start_time):
    """Fase de validação final após o fit."""
    print("\n--- Fase de Validação Final ---")
    model.eval()
    total_loss, preds, targets = compute_val_loss_and_preds(model, dataloader, loss_fn, device, model_name)

    accuracy, f1, loss = compute_val_metrics(preds, targets, total_loss, dataloader)
    print(f"Resultados Finais Fold {kfold+1}: Acurácia: {accuracy:.4f}, F1-Score: {f1:.4f}, Perda: {loss:.4f}")

    df_metrics = update_metrics_df(df_metrics, kfold, accuracy, f1, start_time)
    save_metrics_to_csv(df_metrics, log_dir)
    return df_metrics, preds, targets, f1

## Função `train` Principal

Esta é a função principal que orquestra todo o processo, incluindo o K-fold.

In [ ]:
def train(config, config_path):
    print(f'Iniciando experimento: {config["experiment_name"]}')

    # Define o dispositivo (GPU se disponível)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    # Extrai hiperparâmetros
    learning_rate = float(config["learning_rate"])
    batch_size = config["batch_size"]
    epochs = config["epochs"]
    max_len = config["max_len"]
    model_path = config["model_path"]
    log_dir = config["log_dir"]
    checkpoint_dir = config["checkpoint_dir"]
    model_name = config["model_name"]

    name_arch = model_path.split("-")[0]
    alpha_version = int(config_path.split('/')[-2].split('-')[-1][-1]) # 3, 4 or 5
    flag_twiter = False # Mantido do seu script original

    print(f"Config path: {config_path}")
    if config_path.split('/')[-2].split('-')[0] == "openai":
        dataset_type = "gpt4-openai-classify"
        # (O resto da lógica de twitter foi mantido, mas não será ativado por este config)
    elif config_path.split('/')[-2].split('-')[0] == "deepseek":
        dataset_type = "deepseek"
    else:
        dataset_type = "minigpt4-classify"

    experiment_group = config_path.split('/')[-2].split('-')[-2]
    print(f"Alpha version: {alpha_version} | Experiment group: {experiment_group}")

    df = load_experiment_data(alpha_version, dataset_type, experiment_group)

    if df is None:
        print("Falha ao carregar o dataset. Abortando.")
        return

    print("Preview do Dataset:")
    print(df.head())
    print(f"Classes: {df.sentiment.unique()}")
    print(f"Distribuição:\n{df.sentiment.value_counts()}")

    train_val_df = df.copy()
    train_params = {"batch_size": batch_size, "shuffle": True}
    val_params = {"batch_size": batch_size, "shuffle": False}

    # Correção: O seu script tinha um 'elif' solto, mudei para 'if'
    if model_name == "modern-bert":
        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        df_metrics = pd.DataFrame([])
        best_f1 = 0

        for fold, (train_idx, val_idx) in enumerate(kfold.split(train_val_df)):
            print(f"\n========== FOLD {fold + 1} / 5 ==========")
            print(f"Carregando modelo: {model_path}")
            start_time = time.time()

            model = ModernBERTModel(model_path, len(df.sentiment.unique()))
            tokenizer = AutoTokenizer.from_pretrained(model_path)
            model.to(device)

            optimizer = AdamW(
                model.parameters(),
                lr=learning_rate,
                weight_decay=1e-6
            )

            train_df = train_val_df.iloc[train_idx].reset_index(drop=True)
            val_df = train_val_df.iloc[val_idx].reset_index(drop=True)

            # (A lógica do 'flag_twiter' para o val_df foi mantida aqui)
            if flag_twiter:
                print("Lógica específica do Twitter ativada para validação...")
                # ... (seu código original de carregamento do val_df do twitter) ...
                pass

            # Calcular pesos das classes
            class_size = train_df.sentiment.value_counts().sort_index().to_list()
            class_weights = torch.Tensor([sum(class_size) / c for c in class_size]).to(device) # Normalizado
            print(f"Pesos das classes: {class_weights}")

            train_dl = data_loader(train_df, tokenizer, max_len, train_params)
            val_dl = data_loader(val_df, tokenizer, max_len, val_params)

            model, loss_fn = fit(
                model, class_weights, epochs, optimizer,
                train_dl, val_dl,
                log_dir, checkpoint_dir,
                name_arch, fold, model_name,
                alpha_version, experiment_group,
                device
            )

            df_metrics, y_pred, y_true, f1_val = val(log_dir, model, val_dl, loss_fn, fold, df_metrics, model_name, device, start_time)

            fine_tuning = "finetuned" if log_dir.split('/')[0] != "experiments-not-finetuning" else "not_finetuned"
            current_name_arch = dataset_type + "_modernbert"

            # Salvar o melhor checkpoint *entre os folds*
            if f1_val > best_f1:
                print(f"Novo melhor F1-score global: {f1_val:.4f} (anterior: {best_f1:.4f})")
                best_f1 = save_checkpoint(model, checkpoint_dir, current_name_arch, experiment_group, alpha_version, fine_tuning, f1_val, best_f1)

            # Salvar predições
            result_df = pd.DataFrame({
                "text": val_df["text"].to_list(),
                "target": y_true,
                "prediction": y_pred
            })
            result_df.to_csv(os.path.join(log_dir, f"test_logs_fold_{fold+1:02d}.csv"), index=False)

            del model # Limpar memória da GPU
            torch.cuda.empty_cache()

        # Calcular métricas finais
        mean_f1 = np.mean(df_metrics["f1_score"].to_numpy())
        std_f1 = np.std(df_metrics["f1_score"].to_numpy())

        print("\n========== RESULTADO FINAL (K-Fold) ==========")
        print(f"F1-Score Médio: {mean_f1 * 100:.2f}% ± {std_f1 * 100:.2f}%")
        print("Métricas por fold:")
        print(df_metrics)

    else:
        print(f"Modelo '{model_name}' não suportado por este script.")

## Inicial o TensorBoard

Vamos carregar a extensão do TensorBoard no Colab. Ele ficará monitorando o diretório de logs

In [ ]:
# Carregar a extensão do TensorBoard
%load_ext tensorboard

# Iniciar o TensorBoard
# Ele vai monitorar o diretório de logs que definimos no config.yaml
%tensorboard --logdir experiments-finetuning/openai-modernbert-experiment-p3-alpha3/logs

## Executar o Treinamento

Finalmente, executamos a função `train`. O TensorBoard (célula acima) será atualizada automaticamente à medida que os logs forem escritos durante o treinamento.

In [ ]:
# Definir o path do config (já o criamos)
config_path = "experiments-finetuning/openai-modernbert-experiment-p3-alpha3/config.yaml"

# Carregar o config
config = load_config(config_path)

# Iniciar o treinamento
train(config, config_path)